# Data Extraction

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import praw
from psaw import PushshiftAPI
from datetime import datetime as dt
import time

In [3]:
client_id = "Ijfzy88liEKe7iGMHtKORg"
client_secret = "3kt2_MM5k0AF8PIFXVh32lAr1xEhXg"
user_agent = "Scraper 1.0 by /u/charltonxqh"

In [9]:
def setup_reddit_client(client_id, client_secret, user_agent):
    return praw.Reddit(client_id=client_id, client_secret=client_secret, user_agent=user_agent)

def extract_comments_voo(reddit, subreddit_name, keyword, limit=1000):
    """
    Extract comments from the specified subreddit for submissions containing the keyword 'VOO'.
    Limit the number of comments to 1000.
    """
    subreddit = reddit.subreddit(subreddit_name)
    comments_data = []
    comment_count = 0

    for submission in subreddit.search(keyword, sort='new'):
        # Check if the submission title contains the keyword
        if keyword.lower() in submission.title.lower():
            submission.comments.replace_more(limit=None)
            for comment in submission.comments.list():
                if comment_count >= limit:
                    break

                comment_data = {
                    'author': str(comment.author),
                    'id': comment.id,
                    'created_utc': dt.fromtimestamp(comment.created_utc),
                    'permalink': comment.permalink,
                    'body': comment.body,
                    'score': comment.score,
                    'subreddit': str(comment.subreddit)
                }
                comments_data.append(comment_data)
                comment_count += 1

        if comment_count >= limit:
            break

    return comments_data

In [10]:
# Initialize and extract comments
reddit = setup_reddit_client(client_id, client_secret, user_agent)
comments = extract_comments_voo(reddit, "ETFs", "VOO")
comments_voo_df = pd.DataFrame(comments)

In [11]:
comments_voo_df.head()

,author,id,created_utc,permalink,body,score,subreddit
0,lotterytix,kwh3sji,2024-03-25 20:10:23,/r/ETFs/comments/1bmqbxg/new_to_investing_is_v...,Maybe consider VOO and a mid/small cap value f...,1,ETFs
1,AlgoTradingQuant,kwczgum,2024-03-25 00:51:21,/r/ETFs/comments/1bmoom7/diversifying_my_ira_f...,I’m retired and hold a 100% equities portfolio...,8,ETFs
2,foldinthechhese,kwdbk25,2024-03-25 02:02:08,/r/ETFs/comments/1bmoom7/diversifying_my_ira_f...,The more experienced investors recommend a ble...,5,ETFs
3,SirChetManly,kwd6nto,2024-03-25 01:33:43,/r/ETFs/comments/1bmoom7/diversifying_my_ira_f...,It isn't *risky* by any stretch. You're exclud...,2,ETFs
4,ZAROV8862,kwei3zo,2024-03-25 06:17:54,/r/ETFs/comments/1bmoom7/diversifying_my_ira_f...,Enough said :)),2,ETFs


In [12]:
comments_voo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 932 entries, 0 to 931
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   author       932 non-null    object        
 1   id           932 non-null    object        
 2   created_utc  932 non-null    datetime64[ns]
 3   permalink    932 non-null    object        
 4   body         932 non-null    object        
 5   score        932 non-null    int64         
 6   subreddit    932 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 51.1+ KB


### Save the DataFrame as a CSV File.

In [14]:
comments_voo_df.to_csv('reddit_comment_voo.csv', index=False)